### TYK2 Chemical Footprints

Interactive chemical footprints are located [here](http://radula.octantbio.net:9260/); to access them, you must be on the Octant VPN.

#### Raw Summary Statistics

We read in the unnormalized summary statistics for all IFN-alpha DMS datasets. This includes assays 3, 4, 7, and 8, where Assay 8 only had two protocol versions each of three conditions. The `make_plot()` function constructs the simple confidence interval/forest plot for a given variant (defined as a position and mutant amino acid) across all conditions and assays.

In [7]:
library(plotly)
library(patchwork)
library(ggpubr)
library(furrr)
library(tidyverse)

colors <- c("Neither" = "gray",
            "Both" = "purple",
            "BMS-986202" = "red",
            "Zasocitinib" = "blue")

make_plot <- function(pos_id, aa_id){

    if(!file.exists(str_c("../footprints/varplots/", pos_id, aa_id, ".jpeg"))){
        df <- sumstats_raw %>% filter(pos == pos_id, aa == aa_id)
        the_plot <- df %>%
            ggplot() +
                geom_pointrange(aes(x = condition, y = estimate,
                                    ymin = estimate - 2*std.error,
                                    ymax = estimate + 2*std.error)) +
                geom_hline(yintercept = 0) +
                theme_pubr(base_size = 10) +
                xlab("") + ggtitle(str_c(pos_id, aa_id)) +
                facet_wrap(~assay, scales = "free_y", ncol = 1) +
                coord_flip(ylim = c(-3, 3))
        ggsave(str_c("../footprints/varplots/", pos_id, aa_id, ".jpeg"), the_plot, width = 8, height = 8)
        #return(the_plot)
    }
}

plan(multicore, workers = 40)

sumstats_raw <- bind_rows(
    read_tsv("../sumstats/TYK2/run3/tyk2-assay3-unnormalized.tsv") %>% mutate("assay" = "Assay 3"),
    read_tsv("../sumstats/TYK2/run4/tyk2-assay4-unnormalized.tsv") %>% mutate("assay" = "Assay 4"),
    read_tsv("../sumstats/TYK2/run7/tyk2-assay7-unnormalized.tsv") %>% mutate("assay" = "Assay 7"),
    read_tsv("../sumstats/TYK2/run8/tyk2-assay8-unnormalized.tsv") %>% mutate("assay" = "Assay 8", "condition" = str_c(condition, "_", protocol))
)

Rows: 142338 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): clone, chunk, condition, aa, version
dbl (7): pos, estimate, std.error, statistic, p.value, dispersion, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 142320 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): clone, chunk, condition, aa, version, drug
dbl (8): pos, estimate, std.error, statistic, p.value, dispersion, conc, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 118595 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): clone, chunk, condition, aa, version, drug
dbl (8): pos, estimate

#### Assay 4, Drug Resistance, BMS-986202 and Zasocitinib

For the specific footprint, we read in Assay 4 and retrieve the normalized summary statistics (condition versus none) for Zasocitinib and BMS-986202. We also run the `make_plot` function on all variants from any position with at least one significant variant (if the plot does not already yet exist) using the `future_map2` call:

In [8]:
sumstats_class <- read_tsv("../sumstats/TYK2/run4/tyk2-assay4.tsv") %>%
    filter(condition %in% c("IFNalpha+BMS-986202_1e-06 - None_0", "IFNalpha+Zasocitinib_1e-06 - None_0")) %>%
    select(pos, drug, aa, statistic, p.adj) %>%
    pivot_wider(names_from = drug, values_from = c(statistic, p.adj)) %>%
    mutate(`FDR < 0.01` = case_when(`p.adj_IFNalpha+BMS-986202` < 0.01 & `p.adj_IFNalpha+Zasocitinib` < 0.01 ~ "Both",
                           `p.adj_IFNalpha+BMS-986202` < 0.01 & `p.adj_IFNalpha+Zasocitinib` > 0.01 ~ "BMS-986202",
                           `p.adj_IFNalpha+BMS-986202` > 0.01 & `p.adj_IFNalpha+Zasocitinib` < 0.01 ~ "Zasocitinib",
                           TRUE ~ "Neither"),
          `FDR < 0.01` = if_else(`statistic_IFNalpha+Zasocitinib` < 0 & `statistic_IFNalpha+BMS-986202` < 0,
                                 "Neither",
                                 `FDR < 0.01`))

sig_vars <- sumstats_class %>% filter(`FDR < 0.01` != "Neither") %>% pull(pos) %>% unique()
prepped_data <- sumstats_class %>% filter(pos %in% sig_vars) %>% mutate(id = str_c('../varplots/', pos, aa, '.jpeg'))

sumstats_raw_sig <- sumstats_raw %>% filter(pos %in% sig_vars) %>% select(pos, aa) %>% distinct()

nullout <- future_map2(.x = sumstats_raw_sig$pos, .y = sumstats_raw_sig$aa, ~make_plot(pos_id = .x, aa_id = .y))

Rows: 118600 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): drug, clone, chunk, aa, version, condition
dbl (8): conc, pos, estimate, std.error, dispersion, statistic, p.value, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Finally, we construct the plot with `ggplot` and convert it into an interactive plotly object with `ggplotly`. Then, the Javascript code called by `onRender` processes the path to the variant images created above and displays them in the generated HTML:

In [9]:
tile_plot <- ggplot(prepped_data,
                    aes(customdata = id,
                        text = str_c(pos, aa))) +
        geom_tile(aes(x = as.factor(pos), y = aa, fill = `FDR < 0.01`)) +
        scale_fill_manual(values = colors) +
        xlab("Position") + ylab("") +
        theme(plot.margin = unit(c(1, 15, 1, 1), "cm"))

converted_plot <- ggplotly(tile_plot, width = 1200, height = 600, tooltip = "text") %>%
  htmlwidgets::onRender("
    function(el, x) {
      el.on('plotly_click', function(d) {
        var image_location = d.points[0].data.customdata[0];

        var img = {
          source: image_location,
          x: 1.1,  // Adjust this value to position the image on the right side
          y: 0.95,  // Center the image vertically
          sizex: 1,
          sizey: 1,
          xref: 'paper',
          yref: 'paper'
        };

        Plotly.relayout(el.id, {
            images: [img]
        });
      });
    }
  ")

htmlwidgets::saveWidget(converted_plot, "../footprints/Assay4-Resistance.html", selfcontained = TRUE)